## Import Data

In [104]:
import csv 
import pandas as pd
import numpy as np

In [5]:
pubtator_abtract_entites = pd.read_csv("../results/pubtator_abstract_entities.csv", sep='\t')

In [27]:
pubtator_abtract_entites

,entity,entity-type,pmid
0,hallmark of heart failure,Disease,34806902
1,murine,Species,34806902
2,ADAMTS5,Gene,34806902
3,a disintegrin and metalloproteinase with throm...,Gene,34806902
4,ADAMTS5,Gene,34806902
...,...,...,...
141,transgelin,Gene,29484645
142,human,Species,29133944
143,death,Disease,29133944
144,human,Species,29133944


## Test PMID 34098726

In [33]:
pmid_34098726 = pubtator_abtract_entites[pubtator_abtract_entites['pmid'] == 34098726].groupby('entity-type').get_group('Gene')

In [34]:
pmid_34098726

,entity,entity-type,pmid
36,HINT1,Gene,34098726
37,histidine triad nucleotide-binding protein 1,Gene,34098726
40,HINT1,Gene,34098726
43,Hint1,Gene,34098726
45,HINT1,Gene,34098726
49,Hint1,Gene,34098726
50,cTnT,Gene,34098726
51,HINT1,Gene,34098726
53,Hoxa5,Gene,34098726
54,homeobox A5,Gene,34098726


In [40]:
import requests
import json

In [47]:
test_query = requests.get('https://www.uniprot.org/uniprotkb?query=Hint1')

In [55]:
sm = "https://rest.uniprot.org/uniprotkb/search?query=gene:'histidine triad nucleotide-binding protein 1'"
test_query = requests.get(sm)

In [56]:
test_query.json()

{'results': []}

In [ ]:
# Overall goal: Get the proper uniprot ID
# two cases
    # gene name, protein name
    # if one has no results, do the other one
# specify the organisim type in the query

In [67]:
# it looks like I found a way of getting all queries
# now I need to get the species (maybe save this step for next time)
url = "https://rest.uniprot.org/uniprotkb/search?query="
gene_request = requests.get(url + "gene:'hint1'")
name_request = requests.get(url + "protein_name:'hint1'")
if str(name_request.json()['results']) == str(gene_request.json()['results']):
    print("Life is Easier")
else:
    print(gene_request.json()['results'])
    print()
    print(name_request.json()['results'])

[{'entryType': 'UniProtKB reviewed (Swiss-Prot)', 'primaryAccession': 'P49773', 'secondaryAccessions': ['Q9H5W8'], 'uniProtkbId': 'HINT1_HUMAN', 'entryAudit': {'firstPublicDate': '1996-10-01', 'lastAnnotationUpdateDate': '2023-02-22', 'lastSequenceUpdateDate': '2007-01-23', 'entryVersion': 212, 'sequenceVersion': 2}, 'annotationScore': 5.0, 'organism': {'scientificName': 'Homo sapiens', 'commonName': 'Human', 'taxonId': 9606, 'lineage': ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']}, 'proteinExistence': '1: Evidence at protein level', 'proteinDescription': {'recommendedName': {'fullName': {'evidences': [{'evidenceCode': 'ECO:0000305', 'source': 'PubMed', 'id': '15703176'}], 'value': "Adenosine 5'-monophosphoramidase HINT1"}, 'ecNumbers': [{'evidences': [{'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '15703176'}, {'evidenceCode': 'ECO:0000

In [108]:
# Write a function to find the relevant UniProt ID for all articles
def find_uniprot_id(pmid: str, entities: list, species: str) -> list:
    ids = []

    # save the base url
    url = "https://rest.uniprot.org/uniprotkb/search?query="

    # specify species in query
    if species == 'HUMAN':
        organism_id = '9606'
    for entity in entities:
        # each entity may do better queried either as a gene or protein name
        gene_request = requests.get(url + "gene:" + str(entity) + '+organism_id:' + organism_id)
        name_request = requests.get(url + "protein_name:" + str(entity) + '+organism_id' + organism_id)

        # return the first result of the search
        try:
            if gene_request.json()['results'] != []:
                ids.append((entity, gene_request.json()['results'][0]['primaryAccession']))
            elif name_request.json()['results'] != []:
                ids.append((entity, name_request.json()['results'][0]['primaryAccession']))
            else:
                # print("IN IF gene_request", gene_request.text)
                # print("IN IF name_request", name_request.text)
                ids.append((entity, None))
        except:
            # print("gene_request", gene_request.text)
            # print("name_request", name_request.text)
            ids.append((entity, None))
    
    return ids

In [109]:
inp = np.unique(np.array(pmid_34098726['entity']))
x = find_uniprot_id('34098726', inp, 'HUMAN')

In [110]:
x

[('ERK', 'P29323'),
 ('HINT1', 'P49773'),
 ('HOXA5', 'P20719'),
 ('Hint1', 'P49773'),
 ('Hoxa5', 'P20719'),
 ('MEK', None),
 ('TGF-beta', None),
 ('YY1', 'P25490'),
 ('cTnT', None),
 ('histidine triad nucleotide-binding protein 1', None),
 ('homeobox A5', None)]

For next time: Apply this to the rest of the entites. Maybe do some QC.